## Import

In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import os

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import core.config as cfg
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as pri

In [ ]:
import helpers.s3 as hs3
import vendors2.cme.reader as cmer
import vendors2.kibot.PartTask269_liquidity_analysis_utils as lau
import vendors2.kibot.utils as kut

In [ ]:
# import infra.helpers.telegram_notify.telegram_notify as tg
# tgn = tg.TelegramNotify()

In [ ]:
print(env.get_system_signature())

pri.config_notebook()

# TODO(gp): Changing level during the notebook execution doesn't work. Fix it.
# dbg.init_logger(verbosity=logging.DEBUG)
dbg.init_logger(verbosity=logging.INFO)
# dbg.test_logger()

_LOG = logging.getLogger(__name__)

# Metadata

In [ ]:
df1 = kut.read_1min_contract_metadata()
df1.head(3)

In [ ]:
df2 = kut.read_daily_contract_metadata()
df2.head(3)

In [ ]:
df3 = kut.read_tickbidask_contract_metadata()
df3.head(3)

In [ ]:
df4 = kut.read_continuous_contract_metadata()
print(df4.head(3))

print(df4["Exchange"].unique())

# Load product specs

In [ ]:
product_specs = cmer.read_product_specs()

In [ ]:
product_specs.head()

In [ ]:
product_specs.info()

# Explore metadata

In [ ]:
df4["Exchange"].value_counts()

In [ ]:
df3["Exchange"].value_counts()

Kibot only has the CME group futures.

In [ ]:
product_specs["Globex"].head()

In [ ]:
# daily_futures_w_ext = os.listdir
#    "/data/kibot/All_Futures_Continuous_Contracts_daily/"
# )

daily_futures_path = os.path.join(
    hs3.get_path(), "kibot/All_Futures_Continuous_Contracts_daily"
)
daily_futures_w_ext = hs3.ls(daily_futures_path)

In [ ]:
daily_futures_w_ext[:5]

In [ ]:
daily_futures = list(map(lambda x: x[:-7], daily_futures_w_ext))
daily_futures[:5]

In [ ]:
len(set(daily_futures)), df3["SymbolBase"].nunique()

In [ ]:
np.setdiff1d(df3["SymbolBase"].dropna().values, daily_futures)

In [ ]:
product_specs["Globex"].nunique()

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df3["SymbolBase"].dropna().values
)

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df3["SymbolBase"].dropna().values
).shape

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df2["Symbol"].dropna().values
).shape

In [ ]:
np.intersect1d(
    product_specs["Globex"].dropna().unique(), df1["Symbol"].dropna().values
).shape

In [ ]:
product_specs[product_specs["Globex"].isna()]

In [ ]:
mask = ["GAS" in d or "OIL" in d for d in df4["Description"].astype(str)]
print(sum(mask))
print(df4[mask].drop(["SymbolBase", "Size(MB)"], axis=1))

In [ ]:
df4[mask]["Symbol"].values

# Read config

In [ ]:
config = cfg.Config.from_env()

if config is None:
    config = cfg.Config()
    config_tmp = config.add_subconfig("read_data")
    # config_tmp["nrows"] = 100000
    config_tmp["nrows"] = None
    #
    config["zscore_com"] = 28

print(config)

# Prices

## Read daily prices

In [ ]:
all_symbols = daily_futures.copy()

In [ ]:
symbols = df4[mask]["Symbol"].values
symbols

In [ ]:
file_name = os.path.join(daily_futures_path, "%s.csv.gz")
daily_price_dict_df = kut.read_data(
    "D", "continuous", symbols, nrows=config["read_data"]["nrows"]
)

daily_price_dict_df["CL"].tail(2)

# Top futures by volume

## Sum volume

In [ ]:
daily_vol = lau.get_price_data(daily_price_dict_df, lau.KIBOT_VOL, "sum")
daily_vol.sort_values("sum_vol", ascending=False)

## Mean volume

In [ ]:
mean_vol = lau.get_price_data(daily_price_dict_df, lau.KIBOT_VOL, "mean")
mean_vol.sort_values("mean_vol", ascending=False)

# Study volume

In [ ]:
symbol = "CL"

In [ ]:
daily_prices = lau.read_kibot_prices("D", "continuous", symbol)

In [ ]:
tsds = lau.TimeSeriesDailyStudy(daily_prices[lau.KIBOT_VOL], data_name=symbol)

In [ ]:
tsds.execute()

In [ ]:
minutely_prices = lau.read_kibot_prices("T", "continuous", symbol)

In [ ]:
tsms = lau.TimeSeriesMinuteStudy(minutely_prices[lau.KIBOT_VOL], data_name=symbol)

In [ ]:
tsms.execute()

In [ ]:
five_min_vol = minutely_prices[lau.KIBOT_VOL].resample("5min").sum()
five_min_vol.head()

In [ ]:
tsms_five_min = lau.TimeSeriesMinuteStudy(
    five_min_vol, data_name=symbol, freq_name="Five minute"
)

In [ ]:
tsms_five_min.execute()

In [ ]:
# tgn.notify("Volume study is complete")

## How is the volume related to the open interest from the metadata?

In [ ]:
product_specs.head()

In [ ]:
product_specs[product_specs["Globex"] == symbol]["Open Interest"].values

In [ ]:
product_specs[product_specs["Globex"] == symbol]["Volume"].values

In [ ]:
daily_prices[lau.KIBOT_VOL].max()

In [ ]:
minutely_prices[lau.KIBOT_VOL].max()

# CME mapping

## Groups overview

In [ ]:
pc = lau.ProductSpecs()

In [ ]:
pc.product_specs.info()

In [ ]:
pc.product_specs["Product Group"].value_counts().plot(kind="bar", rot=0)
plt.title("Number of futures for each product group in CME")
plt.show()

In [ ]:
pc.product_specs["Sub Group"].value_counts().plot(kind="bar")
plt.xticks(ha="right", rotation=30, rotation_mode="anchor")
plt.title("Number of futures for each sub group in CME")
plt.show()

In [ ]:
pc.product_specs["Category"].astype(str).value_counts()

In [ ]:
pc.product_specs["Sub Category"].astype(str).value_counts()

## By symbol

In [ ]:
pc.get_metadata_symbol(symbol)

In [ ]:
pc.get_product_group(symbol)

In [ ]:
pc.get_trading_hours(symbol)

## For product group

In [ ]:
energy_symbols = pc.get_symbols_product_group("Energy")
energy_symbols[:4]

In [ ]:
np.intersect1d(energy_symbols, daily_futures)

In [ ]:
np.intersect1d(energy_symbols, daily_futures).shape